In [1]:
from bs4 import BeautifulSoup,Comment
import urllib.request
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
#from pandas import DataFrame as df

In [ ]:
#to get soup for 2020 team offense using Comment, used below
url = "https://www.pro-football-reference.com/years/2020/"
soup = BeautifulSoup(requests.get(url).content, 'html.parser')
table = soup.select_one('#all_team_stats').find_next(text=lambda t: isinstance(t, Comment))
table = BeautifulSoup(table, 'html.parser')



In [ ]:
#what the HTML looks like
table.prettify()[0:2500]

In [7]:
#function to rename some columns and columns that are missing titles from HTML table
def rename_cols(df):
    dict = {'Unnamed: 0_level_0':'TotOff',
            'Unnamed: 1_level_0':'TotOff',
            'Unnamed: 2_level_0':'TotOff',
            'Unnamed: 3_level_0':'TotOff',
            'Unnamed: 4_level_0':'TotOff',
            'Tot Yds & TO':'TotOff',
            'Tot Yds & TO':'TotOff',
            'Tot Yds & TO':'TotOff',
            'Unnamed: 8_level_0':'TotOff',
            'Unnamed: 9_level_0':'TotOff',
            'Unnamed: 25_level_0':'Score%',
            'Unnamed: 26_level_0':'TurnOver%',
            'Unnamed: 27_level_0':'EXP',}
        
    # call rename() method
    df.rename(columns=dict,inplace=True)
    return(df)


In [14]:
#for loop to get data for X years


year = 2020
#URL for website data
url = "https://www.pro-football-reference.com/years/"
urls = []
OFFdfs = []
for index in np.arange(10):
    #create URL for BeautifulSoup
    urls.append(url + str(year - index) +'/')
    years = year - index

count = 0
for index in urls:
    soup = BeautifulSoup(requests.get(index).content, 'html.parser')
    table = soup.select_one('#all_team_stats').find_next(text=lambda t: isinstance(t, Comment))
    table = BeautifulSoup(table, 'html.parser')    
    #create DF from html table, drop last 3 rows of data 
    OFFdf = pd.read_html(str(table))[0]
    curr_year = year - count
    OFFdf['Year'] = curr_year
    OFFdfs.append(OFFdf[:-3])
    count+=1


#initialize Master Team Offense Dataframe    
MasterOFF = OFFdfs[0]

for i in range(len(OFFdfs)-1):    
    #append DF to master, reset index to continuous
    MasterOFF = MasterOFF.append(OFFdfs[i+1],ignore_index = True)

#rename columns to useful index    
MasterOFF = rename_cols(MasterOFF)



In [ ]:
MasterOFF.info()

In [22]:
MasterOFF.head(5)

TotOff                                                                      \
      Rk                    Tm     G     PF     Yds     Ply  Y/P    TO    FL   
0    1.0     Green Bay Packers  16.0  509.0  6224.0   990.0  6.3  11.0   6.0   
1    2.0         Buffalo Bills  16.0  501.0  6343.0  1034.0  6.1  22.0  11.0   
2    3.0  Tampa Bay Buccaneers  16.0  492.0  6145.0  1017.0  6.0  17.0   5.0   
3    4.0      Tennessee Titans  16.0  491.0  6343.0  1031.0  6.2  12.0   5.0   
4    5.0    New Orleans Saints  16.0  482.0  6023.0  1045.0  5.8  17.0   9.0   

          ... Rushing             Penalties               Score% TurnOver%  \
    1stD  ...      TD  Y/A   1stD       Pen     Yds 1stPy    Sc%       TO%   
0  358.0  ...    16.0  4.8  114.0      84.0   684.0  28.0   49.7       5.6   
1  397.0  ...    16.0  4.2  119.0     102.0   941.0  38.0   49.4      11.8   
2  364.0  ...    16.0  4.1   82.0      84.0   715.0  44.0   47.8       8.9   
3  381.0  ...    26.0  5.2  142.0      86.0   783.0  36.0   47.9       7.2   
4  367.0  ...    30.0  4.6  147.0      98.0  1005.0  21.0   45.5       9.0   

      EXP  Year  
      EXP        
0  290.75  2020  
1  228.66  2020  
2  246.24  2020  
3  245.01  2020  
4  176.94  2020  

[5 rows x 29 columns]

In [ ]:
MasterOFF.keys()

In [ ]:
plt.figure(figsize=(10, 10))
#pointsfor = MasterOFF.TotOff.PF
MasterOFF.plot.scatter("Year", [('TotOff','PF')] , figsize=(15, 10))
plt.show()

In [21]:
#Get Team Defense data

year = 2020
#number of years
n=10
#URL for website data
url = "https://www.pro-football-reference.com/years/"
urls = []
DEFdfs = []
ADV_DEFdf = []
Team_Sacks = pd.DataFrame(['Tm','Sk'])
for index in np.arange(3):
    #create URL for BeautifulSoup
    urls.append(url + str(year - index) +'/opp.htm')
    years = year - index

    
#mask for advanced_defense table data
sacks = ['Tm','Sk']

#team_stats table does not include Sacks, so need advanced_defense table data and merge tables.
count = 0
for index in urls:
    soup = BeautifulSoup(requests.get(index).content, 'html.parser')
    table = soup.find('table', id = 'team_stats') 
    table2 = soup.find('table', id = 'advanced_defense')
    #create DF from html table, drop last 3 rows of data 
    DEFdf = pd.read_html(str(table))[0]
    DEFdf[-3:]
    #rename each DF
    rename_cols(DEFdf)
    #sort alphabetically
    DEFdf.sort_values([('TotOff', 'Tm')],inplace=True)
    ADV_DEFdf = pd.read_html(str(table2))[0]
    #apply mask
    Team_Sacks = ADV_DEFdf[sacks].copy()
    #sort alphabetically
    Team_Sacks.sort_values(['Tm'],inplace = True)
    #intermediate step prior to appending data. merges team_stats and team sacks data 
    DEFdf_interm = pd.merge(DEFdf,Team_Sacks, left_on =[('TotOff', 'Tm')], right_on = 'Tm' )
    #append to list of team defense data
    curr_year = year - count
    DEFdf_interm['Year'] = curr_year    
    DEFdfs.append(DEFdf_interm)
    count+=1

#add in the year column
#for index in np.arange(10):
    #add Year to DF columns
#    curr_year = year - index
#    DEFdfs[index]['Year'] = curr_year

#initialize master DataFrame with first list of Team Defense data
MasterDEF = DEFdfs[0]

for i in range(len(DEFdfs)-1):    
    #append DF to master, reset index to continuous
    MasterDEF = MasterDEF.append(DEFdfs[i+1],ignore_index = True)


/Users/scottstack/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/scottstack/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/scottstack/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [18]:
MasterDEF.head(5)


,"(TotOff, Rk)","(TotOff, Tm)","(TotOff, G)","(TotOff, PF)","(TotOff, Yds)","(TotOff, Ply)","(TotOff, Y/P)","(TotOff, TO)","(TotOff, FL)","(TotOff, 1stD)",...,"(Rushing, 1stD)","(Penalties, Pen)","(Penalties, Yds)","(Penalties, 1stPy)","(Score%, Sc%)","(TurnOver%, TO%)","(EXP, EXP)",Tm,Sk,Year
0,12.0,Arizona Cardinals,16.0,367.0,5631.0,1054.0,5.3,21.0,10.0,363.0,...,118.0,104.0,841.0,38.0,37.2,10.5,-82.04,Arizona Cardinals,48,2020
1,19.0,Atlanta Falcons,16.0,414.0,6374.0,1034.0,6.2,21.0,9.0,367.0,...,97.0,91.0,800.0,31.0,42.0,12.1,-148.58,Atlanta Falcons,29,2020
2,2.0,Baltimore Ravens,16.0,303.0,5276.0,1017.0,5.2,22.0,12.0,347.0,...,96.0,77.0,646.0,38.0,30.0,12.4,-17.69,Baltimore Ravens,39,2020
3,16.0,Buffalo Bills,16.0,375.0,5640.0,1025.0,5.5,26.0,11.0,351.0,...,125.0,86.0,728.0,27.0,39.4,15.3,-123.10,Buffalo Bills,38,2020
4,18.0,Carolina Panthers,16.0,402.0,5761.0,1022.0,5.6,22.0,15.0,360.0,...,114.0,85.0,713.0,35.0,46.0,12.4,-125.54,Carolina Panthers,29,2020


In [ ]:
MasterDEF.info()

In [ ]:
MasterDEF.keys()